# 02 - Document Parsing: Extract Text, Tables, and Figures

This notebook parses 10-K filings into structured components while preserving table structure and metadata.

**Objectives:**
- Parse PDFs/HTML into text, tables, and figures
- Preserve table structure (headers, units, row/column relationships)
- Extract metadata (company, year, section, page numbers)
- Create table row-sentences for searchability
- Handle OCR for figures when needed

In [13]:
import os
import sys
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import pickle

sys.path.append(str(Path.cwd().parent / 'src'))

from utils.config import RAW_DATA_DIR, PARSED_DATA_DIR
from parsers.filing_parser import FilingParser
from parsers.table_parser import TableParser
from parsers.section_extractor import SectionExtractor

## 1. Load Download Metadata

In [14]:
# Load the download log from previous notebook
download_log = pd.read_csv(RAW_DATA_DIR / 'download_log.csv')
successful_downloads = download_log[download_log['status'] == 'success']

print(f"Total filings to parse: {len(successful_downloads)}")
successful_downloads.head()

Total filings to parse: 1481


,ticker,cik,fiscal_year,file_format,file_path,download_date,status
0,MMM,66740,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
1,MMM,66740,2023,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
2,MMM,66740,2022,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
3,AOS,91142,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:28.987291,success
4,AOS,91142,2023,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:28.987291,success


## 2. Initialize Parsers

In [15]:
# Initialize parsing components
filing_parser = FilingParser()
table_parser = TableParser()
section_extractor = SectionExtractor()

# Sections of interest for 10-K analysis
TARGET_SECTIONS = [
    "Item 1. Business",
    "Item 1A. Risk Factors",
    "Item 7. Management's Discussion and Analysis",
    "Item 8. Financial Statements",
    "Item 8. Consolidated Financial Statements",
    "Notes to Financial Statements",
    "Notes to Consolidated Financial Statements"
]

## 3. Parse Filings

In [ ]:
# Parse each filing
parsed_documents = []
parsing_errors = []

for idx, row in tqdm(successful_downloads.iterrows(), total=len(successful_downloads), desc="Parsing filings"):
    ticker = row['ticker']
    fiscal_year = row['fiscal_year']
    file_path = row['file_path']
    file_format = row['file_format']
    
    try:
        print(f"\nParsing {ticker} {fiscal_year} ({file_format})...")
        
        # Parse the document
        parsed_doc = filing_parser.parse(
            file_path=file_path,
            file_format=file_format,
            ticker=ticker,
            fiscal_year=fiscal_year
        )
        
        # Extract sections
        sections = section_extractor.extract_sections(
            parsed_doc,
            target_sections=TARGET_SECTIONS
        )
        
        # Parse tables with structure preservation
        tables = table_parser.extract_tables(
            parsed_doc,
            preserve_structure=True
        )
        
        # Create table row-sentences for searchability
        table_sentences = table_parser.create_row_sentences(tables)
        
        # Compile parsed document
        parsed_document = {
            'metadata': {
                'ticker': ticker,
                'fiscal_year': fiscal_year,
                'file_format': file_format,
                'file_path': file_path,
                'num_pages': parsed_doc.get('num_pages'),
                'num_sections': len(sections),
                'num_tables': len(tables),
            },
            'sections': sections,
            'tables': tables,
            'table_sentences': table_sentences,
            'figures': parsed_doc.get('figures', []),
        }
        
        # Save parsed document
        output_dir = PARSED_DATA_DIR / ticker
        output_dir.mkdir(parents=True, exist_ok=True)
        
        output_file = output_dir / f"{ticker}_{fiscal_year}_parsed.pkl"
        with open(output_file, 'wb') as f:
            pickle.dump(parsed_document, f)
        
        parsed_documents.append({
            'ticker': ticker,
            'fiscal_year': fiscal_year,
            'parsed_file': str(output_file),
            'num_sections': len(sections),
            'num_tables': len(tables),
            'num_table_sentences': len(table_sentences),
            'status': 'success'
        })
        
        print(f"  ✓ Extracted {len(sections)} sections, {len(tables)} tables")
        
    except Exception as e:
        print(f"  ✗ Error parsing {ticker} {fiscal_year}: {str(e)}")
        parsing_errors.append({
            'ticker': ticker,
            'fiscal_year': fiscal_year,
            'error': str(e),
            'status': 'error'
        })

Parsing filings:   0%|          | 0/1481 [00:00<?, ?it/s]


Parsing MMM 2024 (html)...


Parsing filings:   0%|          | 1/1481 [00:01<25:01,  1.01s/it]

Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 30 columns passed, passed data had 10 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 6 columns passed, passed data had 2 columns
Error parsing table HTML: 6 columns passed, passed data had 2 columns
Error parsing table HTML: 27 columns passed, passed data had 9 columns
Error parsing table HTML: 27 columns passed, passed data had 9 columns
Error parsing table HTML: 27 columns passed, passed data had 15 columns
Error parsing table HTML: 16 columns passed, passed data had 10 columns
  ✓ Extracted 10 sections, 10 tables

Parsing MMM 2023 (html)...


Parsing filings:   0%|          | 2/1481 [00:02<27:59,  1.14s/it]

...

Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 9 columns passed, passed data had 3 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 30 columns passed, passed data had 10 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 12 columns passed, passed data had 4 columns
  ✓ Extracted 10 sections, 10 tables

Parsing ZTS 2023 (html)...


Parsing filings: 100%|█████████▉| 1480/1481 [37:27<00:00,  1.01it/s]

Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 9 columns passed, passed data had 3 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 30 columns passed, passed data had 10 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 12 columns passed, passed data had 4 columns
  ✓ Extracted 10 sections, 10 tables

Parsing ZTS 2022 (html)...


Parsing filings: 100%|██████████| 1481/1481 [37:28<00:00,  1.52s/it]

Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 9 columns passed, passed data had 3 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 30 columns passed, passed data had 10 columns
Error parsing table HTML: 3 columns passed, passed data had 1 columns
Error parsing table HTML: 15 columns passed, passed data had 5 columns
Error parsing table HTML: 12 columns passed, passed data had 4 columns
  ✓ Extracted 10 sections, 10 tables


## 4. Sample Table Structure Analysis

In [17]:
# Inspect a sample parsed table
if parsed_documents:
    sample_doc_path = parsed_documents[0]['parsed_file']
    with open(sample_doc_path, 'rb') as f:
        sample_doc = pickle.load(f)
    
    print("=== Sample Document Structure ===")
    print(f"Ticker: {sample_doc['metadata']['ticker']}")
    print(f"Fiscal Year: {sample_doc['metadata']['fiscal_year']}")
    print(f"Sections: {sample_doc['metadata']['num_sections']}")
    print(f"Tables: {sample_doc['metadata']['num_tables']}")
    
    if sample_doc['tables']:
        print("\n=== Sample Table ===")
        sample_table = sample_doc['tables'][0]
        print(f"Table ID: {sample_table.get('table_id')}")
        print(f"Section: {sample_table.get('section')}")
        print(f"Caption: {sample_table.get('caption', 'N/A')}")
        print(f"Dimensions: {sample_table['data'].shape if 'data' in sample_table else 'N/A'}")
        
        # Display table structure
        if 'data' in sample_table:
            display(sample_table['data'].head())
    
    if sample_doc['table_sentences']:
        print("\n=== Sample Table Sentences (first 5) ===")
        for i, sent in enumerate(sample_doc['table_sentences'][:5]):
            print(f"{i+1}. {sent['sentence']}")
            print(f"   Source: Table {sent['table_id']}, Row {sent['row_idx']}\n")

=== Sample Document Structure ===
Ticker: MMM
Fiscal Year: 2024
Sections: 10
Tables: 10

=== Sample Table ===
Table ID: T1
Section: Unknown
Caption: 
Dimensions: (0, 0)


""


## 5. Parsing Summary and Statistics

In [18]:
# Create parsing summary
parsing_df = pd.DataFrame(parsed_documents)
errors_df = pd.DataFrame(parsing_errors)

# Save logs
parsing_df.to_csv(PARSED_DATA_DIR / 'parsing_log.csv', index=False)
if len(errors_df) > 0:
    errors_df.to_csv(PARSED_DATA_DIR / 'parsing_errors.csv', index=False)

print("\n=== Parsing Summary ===")
print(f"Successfully parsed: {len(parsing_df)}")
print(f"Parsing errors: {len(errors_df)}")
print(f"Success rate: {len(parsing_df) / len(successful_downloads) * 100:.2f}%")

if len(parsing_df) > 0:
    print("\n=== Statistics ===")
    print(f"Total sections extracted: {parsing_df['num_sections'].sum()}")
    print(f"Total tables extracted: {parsing_df['num_tables'].sum()}")
    print(f"Total table sentences: {parsing_df['num_table_sentences'].sum()}")
    
    print("\n=== Per-Document Averages ===")
    print(f"Avg sections per document: {parsing_df['num_sections'].mean():.2f}")
    print(f"Avg tables per document: {parsing_df['num_tables'].mean():.2f}")
    print(f"Avg table sentences per document: {parsing_df['num_table_sentences'].mean():.2f}")

parsing_df.head()


=== Parsing Summary ===
Successfully parsed: 1481
Parsing errors: 0
Success rate: 100.00%

=== Statistics ===
Total sections extracted: 14810
Total tables extracted: 14740
Total table sentences: 15838

=== Per-Document Averages ===
Avg sections per document: 10.00
Avg tables per document: 9.95
Avg table sentences per document: 10.69


,ticker,fiscal_year,parsed_file,num_sections,num_tables,num_table_sentences,status
0,MMM,2024,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,10,10,0,success
1,MMM,2023,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,10,10,0,success
2,MMM,2022,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,10,10,0,success
3,AOS,2024,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,10,10,0,success
4,AOS,2023,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,10,10,0,success


## 6. Quality Checks

In [19]:
# Check for documents with suspiciously low content
print("=== Quality Checks ===")

low_section_docs = parsing_df[parsing_df['num_sections'] < 3]
if len(low_section_docs) > 0:
    print(f"\nWarning: {len(low_section_docs)} documents with < 3 sections:")
    print(low_section_docs[['ticker', 'fiscal_year', 'num_sections']])

low_table_docs = parsing_df[parsing_df['num_tables'] < 5]
if len(low_table_docs) > 0:
    print(f"\nWarning: {len(low_table_docs)} documents with < 5 tables:")
    print(low_table_docs[['ticker', 'fiscal_year', 'num_tables']])

# Check for missing key sections
print("\n=== Section Coverage Check ===")
for doc_info in parsed_documents[:5]:  # Sample first 5
    with open(doc_info['parsed_file'], 'rb') as f:
        doc = pickle.load(f)
    
    section_titles = [s['title'] for s in doc['sections']]
    print(f"\n{doc['metadata']['ticker']} {doc['metadata']['fiscal_year']}:")
    for target in TARGET_SECTIONS:
        found = any(target.lower() in title.lower() for title in section_titles)
        status = "✓" if found else "✗"
        print(f"  {status} {target}")

=== Quality Checks ===

    ticker  fiscal_year  num_tables
350    COP         2022           0
532     EG         2023           0
632    GIS         2024           0
633    GIS         2023           0
634    GIS         2022           0
671   HSIC         2024           0
672   HSIC         2023           0

=== Section Coverage Check ===

MMM 2024:
  ✗ Item 1. Business
  ✗ Item 1A. Risk Factors
  ✗ Item 7. Management's Discussion and Analysis
  ✗ Item 8. Financial Statements
  ✗ Item 8. Consolidated Financial Statements
  ✗ Notes to Financial Statements
  ✗ Notes to Consolidated Financial Statements

MMM 2023:
  ✗ Item 1. Business
  ✗ Item 1A. Risk Factors
  ✗ Item 7. Management's Discussion and Analysis
  ✗ Item 8. Financial Statements
  ✗ Item 8. Consolidated Financial Statements
  ✗ Notes to Financial Statements
  ✗ Notes to Consolidated Financial Statements

MMM 2022:
  ✗ Item 1. Business
  ✗ Item 1A. Risk Factors
  ✗ Item 7. Management's Discussion and Analysis
  ✗ Item 8. Fin

## Next Steps

Proceed to **03_indexing.ipynb** to create searchable indices for text and table content.